In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
data = pd.read_csv('clickstream.csv')
ids = pd.read_csv('ab_save_card_2019-09-02_2019-09-30.csv')

In [28]:
data.head()

,client_id,timestamp,event,event_metadata,card_type_used,cart
0,796175212,2018-07-01 00:10:10.356057239,main,{'source: organic'},NaN,NaN
1,796175212,2018-07-01 00:10:11.272860768,onboarding_main,NaN,NaN,NaN
2,796175212,2018-07-01 00:10:17.112819955,main,NaN,NaN,NaN
3,796175212,2018-07-01 00:10:19.543343496,lost,NaN,NaN,NaN
4,419515020,2018-07-01 00:33:47.067460089,products,{'source: facebook'},NaN,NaN


### 1) Число пользователей

Выделим первые события каждого пользователя, так мы оставим уникальных пользователей и при этом исходя из структуры событий, первое событие содержит event_metadata источник трафика

In [39]:
uniUserData = data.copy()
uniUserData.drop_duplicates(subset ='client_id',keep = 'first', inplace = True)
uniUserData['client_id'].value_counts()

403965953    1
504771219    1
291680899    1
44731012     1
566632224    1
            ..
829584990    1
272948479    1
851715758    1
163834742    1
818153471    1
Name: client_id, Length: 192182, dtype: int64

Всего пользователей 192182  
Выделим пользователей группы A и B

In [50]:
uniDataA = uniUserData[~uniUserData['client_id'].isin(ids['client_id'])].copy()
uniDataA.count()

client_id         183833
timestamp         183833
event             183833
event_metadata    183833
card_type_used         0
cart                   0
dtype: int64

In [51]:
uniDataB = uniUserData[uniUserData['client_id'].isin(ids['client_id'])].copy()
uniDataB.count()

client_id         8349
timestamp         8349
event             8349
event_metadata    8349
card_type_used       0
cart                 0
dtype: int64

Пользователей группы B оказалось всего 8349, против 192182 - 8349 = 183833 , что-то явно пошло не так, число пользователей неравное

### 2) Случайность выбора

Попробуем проверить распределение по источникам

In [63]:
uniDataA['event_metadata'].value_counts()

{'source: google'}      56319
{'source: yandex'}      51549
{'source: facebook'}    42563
{'source: organic'}     33402
Name: event_metadata, dtype: int64

google ~ 31%  
yandex ~ 28%  
facebook ~ 23%  
organic ~ 18%  

In [54]:
uniDataB['event_metadata'].value_counts()

{'source: google'}      2464
{'source: yandex'}      2459
{'source: facebook'}    1961
{'source: organic'}     1465
Name: event_metadata, dtype: int64

google ~ 30%  
yandex ~ 30%  
facebook ~ 24%  
organic ~ 18%  
  
Да, пользователей яндекса практически как пользователей гугла, что не есть хорошо, однако наша выборка в 8000 мала, поэтому это может быть статистической погрешностью, попробуем посмотреть что происходит с timestamp  
  
Вычислим начальное и конечное время прихода пользователя и среднее

In [61]:
min_date = uniDataA['timestamp'].min()
max_date = uniDataA['timestamp'].max()
uniDataA['date_var'] = pd.to_datetime(uniDataA['timestamp'])
mean_date = pd.Timestamp(np.nanmean([tsp.value for tsp in uniDataA['date_var']]))
print(min_date)
print(max_date)
print(mean_date)

print('\n')
min_dateB = uniDataB['timestamp'].min()
max_dateB = uniDataB['timestamp'].max()
# mean_dateB = uniDataB['timestamp'].mean()
uniDataB['date_var'] = pd.to_datetime(uniDataB['timestamp'])
mean_dateB = pd.Timestamp(np.nanmean([tsp.value for tsp in uniDataB['date_var']]))
print(min_dateB)
print(max_dateB)
print(mean_dateB)

2018-07-01 00:10:10.356057239
2019-12-31 23:59:15.867960294
2019-03-28 22:28:22.681803520


2018-07-01 21:28:17.404653940
2019-09-30 23:55:45.038700637
2019-08-05 21:13:39.164142336


Среднее по группе B сильно ближе к правой границе своего интервала и распределено по августу-сентябрю, что позволяет нам сделать вывод что данные выбраны грубо и не случайно